# Data

In [1]:
# Pull assets
!git submodule update --init --recursive

### Process MIDI files
Each MIDI file will be converted into a sequence of events as described in the paper, and saved as a npy file.

In [2]:
from utilities.preprocess import preprocess
# Use `aug` to turn on/off data augmentation, including pitch transposition and time scaling.
preprocess(style='CSQ', aug=True)

100%|██████████| 71/71 [01:46<00:00,  1.50s/it]


### Plot the processed sequence

In [1]:
from utilities.midi_io import MIDI
from models import CPI_TIME_QUANTIZATION

mid = MIDI(
    time_quantization=CPI_TIME_QUANTIZATION,
    time_unit="time",
    filename="scratch/e0_fake_generated_piece_10_0.mid"
)
seq = mid.process(ignore_velocity=False, merge_tracks=False, pitch_shift=12, time_scale=89/120)

mid2 = MIDI(
    # "scratch/e0_fake_generated_piece_10_0.mid",
    time_quantization=CPI_TIME_QUANTIZATION,
    time_unit="time"
)
pm = mid2.to_midi(seq)
pm.write("scratch/out.mid")
